In [ ]:
# 1. It begains by identifying the keywords to understand the context of the content. In this step, it avoids unnecessary words such as stop words
# 2. Then it finds the same kind of context in other content to find the similarities. To determine the similarities between two or more contents, the content-based method uses cosine similarity
# 3. It finds similarities by analyzing the correlation between two or more users
# 4. Then finally it generates recommendations by calculating the weighted average of all user ratings for active users

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


In [5]:
movies = pd.read_csv('movies_metadata.csv')

C:\Users\mdani\AppData\Local\Temp\ipykernel_16888\2992503378.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv('movies_metadata.csv')


In [6]:
movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [ ]:
# Now I´m going to implement all of the steps I talked about in the content-based filtering process meentioned above using Python
# Here I will prepare the data first, then select the columns that we will use to understand the context of the content
# then we will remove the stop words and finally, we will find the cosine similarities to generate recommendations

In [9]:
tfidf = TfidfVectorizer(stop_words='english')

In [10]:
movies['overview'] = movies['overview'].fillna('')

In [11]:
overview_matrix = tfidf.fit_transform(movies['overview'])

In [12]:
similarity_matrix = linear_kernel(overview_matrix, overview_matrix)

In [13]:
mapping = pd.Series(movies.index, index= movies['title'])

In [14]:
print(mapping)

title
Toy Story                          0
Jumanji                            1
Grumpier Old Men                   2
Waiting to Exhale                  3
Father of the Bride Part II        4
                               ...  
Subdue                         45461
Century of Birthing            45462
Betrayal                       45463
Satan Triumphant               45464
Queerama                       45465
Length: 45466, dtype: int64


In [ ]:
# Now let´s create a function and have a look at how the recommendation system is working

In [15]:
def recommend_movies(movie_input):
    
    movie_index = mapping[movie_input]
    
    similarity_score = list(enumerate(similarity_matrix[movie_index]))
    
    similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
    
    similarity_score = similarity_score[1:15]
    
    movie_indices = [i[0] for i in similarity_score]
    
    return (movies['title'].iloc[movie_indices])


In [17]:
print(recommend_movies('Avatar'))

23630                       Beware of Pity
3657                     Project Moon Base
27481                      Gregory Go Boom
18002                The War of the Robots
15706                         The American
18444                            Apollo 18
16068                        Bloodbrothers
19943            Welcome to the Space Show
27512    Vengeance: The Story of Tony Cimo
35666        OSS 117: Mission for a Killer
2458                            The Matrix
28645                 Stand by Me Doraemon
29992                            Shakedown
13887                 The Inhabited Island
Name: title, dtype: object
